In [36]:
!pip install konlpy

In [37]:
!pip install demoji

In [38]:
import konlpy
import pandas as pd

# 이모티콘 제거
import demoji
from konlpy.tag import Mecab, Kkma, Hannanum, Okt, Komoran

##유틸리티 함수 제작

###존댓말 토큰 parsing 함수

In [4]:
def getHonorificTokensFromTXT(txt_file_path : str):
  file = open(txt_file_path, 'r')
  data = file.read()
  file.close()
  honorific_tokens_string = data.split('\n')
  honorific_tokens=[]
  for string_token in honorific_tokens_string :
    tokens = string_token.split(',')
    honorific_tokens.append((tokens[0][2:-1], tokens[1][2:-2]))
  return honorific_tokens

###존댓말 토큰 수를 센 df를 생성하는 함수

In [25]:
def getAnalyzedDataFrame(df, tokenizer, honorific_tokens) :
  tokenized = []
  honorific_counts = []
  for sentence in df['Sentence']:
    # emoji 제거
    processed_sentence = demoji.replace(sentence, "")

    # tokenized 된 string을 반환하는 코드
    morph_tokens = tokenizer.morphs(processed_sentence)
    tokenized_string = " ".join(morph_tokens)
    tokenized.append(tokenized_string)

    # 존댓말 토큰의 개수를 세는 코드
    pos_tokens = tokenizer.pos(processed_sentence)
    honorific_count = 0
    for token in pos_tokens:
      if token in honorific_tokens :
        honorific_count += 1
    honorific_counts.append(honorific_count)
  
  df['tokenized'] = tokenized
  df['honorific'] = honorific_counts

  return df
      
  


## 데이터셋에 적용

존댓말 토큰 목록을 받아와서 parsing하고, 단발성 데이터셋을 받아온 뒤, 존댓말 토큰 개수를 센다.

In [20]:
talk_df = pd.read_csv('/content/drive/MyDrive/와이빅타/Sangsubot_NLP/korean.csv', encoding='utf-8')
talk_df.dropna(inplace=True)

###코모란 분석기 케이스

In [26]:
komoran_honorific_tokens = getHonorificTokensFromTXT('/content/drive/MyDrive/와이빅타/Sangsubot_NLP/komoran_honorific_token.txt')
print(komoran_honorific_tokens)

[('저', 'NP'), ('전', 'MM'), ('절', 'NNG'), ('제가', 'NNP'), ('고마워요', 'NNP'), ('해주시', 'NNP'), ('분', 'NNB'), ('어딨어요?', 'NA'), ('님', 'XSN'), ('께', 'JKB'), ('에요', 'EF'), ('에요', 'EC'), ('아요', 'EC'), ('아요', 'EF'), ('예요', 'EF'), ('예요', 'EC'), ('어서요', 'EF'), ('어서요', 'EC'), ('요', 'JX'), ('요', 'EC'), ('요', 'EF'), ('군요', 'EC'), ('군요', 'EF'), ('ㄴ데요', 'EC'), ('어요', 'EF'), ('어요', 'EC'), ('구요', 'EF'), ('구요', 'EC'), ('ㄹ게요', 'EC'), ('게요', 'EC'), ('ㄹ께요', 'EF'), ('지요', 'EF'), ('지요', 'EC'), ('네요', 'EC'), ('네요', 'EF'), ('죠', 'EF'), ('죠', 'EC'), ('니까요', 'EF'), ('니까요', 'EC'), ('ㅂ니까', 'EF'), ('나요', 'EF'), ('ㅂ니다', 'EF'), ('ㅂ니다', 'EC'), ('ㅂ시다', 'EF'), ('ㅂ시다', 'EC'), ('습니다', 'EF'), ('습니다', 'EC'), ('답니다', 'EF'), ('답니다', 'EC'), ('시', 'EP'), ('드리', 'VV'), ('으시', 'EP'), ('ㄴ데요', 'EC'), ('ㄴ걸요', 'EC'), ('ㄹ까요', 'EC'), ('ㄹ까요', 'EF'), ('던가요', 'EF'), ('거든요', 'EC'), ('거든요', 'EF'), ('ㄴ가요', 'EC'), ('더라구요', 'EF'), ('더라구요', 'EC'), ('더라고요', 'EC'), ('더라고요', 'EF'), ('려고요', 'EF'), ('을까요', 'EF'), ('을까요', 'EC'), ('면서요', 'EF'), ('래요', 'EC

In [27]:
komoran_tokenizer = Komoran()
komoran_df = getAnalyzedDataFrame(talk_df, komoran_tokenizer, komoran_honorific_tokens)
komoran_df.to_csv('./komoran.csv')

###분석기 형태소 분석능력 테스트

In [40]:
okt = Okt()
kkma = Kkma()
komoran = Komoran()
hanna = Hannanum()

In [41]:
text = '저를 제자로 받아주세요.'

In [42]:
print(okt.pos(text))
print(kkma.pos(text))
print(komoran.pos(text))
print(hanna.pos(text))

[('저', 'Noun'), ('를', 'Josa'), ('제자', 'Noun'), ('로', 'Josa'), ('받아주세요', 'Verb'), ('.', 'Punctuation')]
[('저', 'NP'), ('를', 'JKO'), ('제자', 'NNG'), ('로', 'JKM'), ('받', 'VV'), ('아', 'ECD'), ('주', 'VXV'), ('세요', 'EFN'), ('.', 'SF')]
[('저', 'NP'), ('를', 'JKO'), ('제자', 'NNG'), ('로', 'JKB'), ('받', 'VV'), ('아', 'EC'), ('주', 'VX'), ('시', 'EP'), ('어요', 'EF'), ('.', 'SF')]
[('저', 'N'), ('를', 'J'), ('제자', 'N'), ('로', 'J'), ('받', 'P'), ('아', 'E'), ('주', 'P'), ('세', 'E'), ('요', 'J'), ('.', 'S')]
